In [2]:
pip install scikit-learn==1.3.2 imbalanced-learn==0.11.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 3.4 MB/s  0:00:02m0:00:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.1
    Uninstalling scikit-learn-1.7.1:
      Successfully uninstalled scikit-learn-1.7.1
  Attempting uninstall: imbalanced-learn━━━━━━━━ 0/2 [scikit-learn]
    Found existing installation: imbalanced-learn 0.14.02 [scikit-learn]
    Uninstalling imbalanced-learn-0.14.0:━━━ 0/2 [scikit-learn]
      Successfully uninstalled imbalanced-learn-0.14.00/2 [scikit-learn]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [imbalanced-learn][imbalanced-learn]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

/Users/aagnasamhita/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [5]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.956043956043956,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.915995115995116,
  'support': 300.0}}

In [6]:
import mlflow

In [7]:
mlflow.set_experiment("Log_reg")
#so it knows which url to track 
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({   #the output of model training , can be dict or individual metrics
        'accuracy':report_dict['accuracy'],
        'recall_class_0':report_dict['0']['recall'],
        'recall_class_1':report_dict['1']['recall'],
        'f1_score_macor':report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr , "Logistic regression")

2025/09/03 16:17:48 INFO mlflow.tracking.fluent: Experiment with name 'Log_reg' does not exist. Creating a new experiment.


RestException: RESOURCE_DOES_NOT_EXIST: Could not find experiment with ID 642048077882516713

In [8]:
#comparing models 


import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [11]:
#Experiment 1: Train Logistic Regression Classifier

log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [12]:
#Experiment 2: Train Random Forest Classifier

rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



In [13]:
#Experiment 3: Train XGBoost

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [18]:
pip install -U scikit-learn imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 6.4 MB/s  0:00:01 eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2 0/2 [scikit-learn]
  Attempting uninstall: imbalanced-learn━━━━━━━━ 0/2 [scikit-learn]
    Found existing installation: imbalanced-learn 0.11.02 [scikit-learn]
    Uninstalling imbalanced-learn-0.11.0:━━━ 0/2 [scikit-learn]
      Successfully uninstalled imbalanced-learn-0.11.00/2 [scikit-learn]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [imbalanced-learn][imbalanced-learn]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [14]:
#Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [15]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



### Track Experiments Using MLFlow

In [16]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [17]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [19]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669724,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.9132610508757297,
   'support': 300.0}},
 {'0': {'precision': 0.9676258992805755,
   'recall': 0.9962962962962963,
   'f1-score': 0.9817518248175183,
   'support': 270.0},
  '1': {'precision': 0.9545454545454546,
   'recall': 0.7,
   'f1-score': 0.8076923076923077,
   'support': 30.0},
  'accuracy': 0.9666666666666667,
  'macro avg': {'precision': 0.961085676913015,
   'recall': 0.8481481481481481,
   'f1-score': 0.894722066254913,
   'support': 300.0},
  'weighted avg': {'precision': 0.966

In [25]:
import mlflow.sklearn
import mlflow.xgboost

In [26]:
import mlflow 
mlflow.set_experiment("anomaly detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_param("model", model_name)

        # Log metrics safely
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report.get('1', {}).get('recall', 0))
        mlflow.log_metric('recall_class_0', report.get('0', {}).get('recall', 0))
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])
        
        # Log model
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")
    

2025/09/03 16:48:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/03 16:49:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/03 16:49:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/739961409105596335/runs/51f68ec89e4a4975bfe661b8e6e21fd1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/739961409105596335


2025/09/03 16:49:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/03 16:49:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/739961409105596335/runs/634d869ef872456d883037cc5d377b09
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/739961409105596335


2025/09/03 16:49:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/03 16:49:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/739961409105596335/runs/8d238a4879084a05a34a8379fbb2a497
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/739961409105596335


2025/09/03 16:49:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/739961409105596335/runs/aa7c60a72db34d358f94a642831221b2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/739961409105596335


In [32]:
#register the model 
model_name="XGB-Smote"
run_id=input("enter run id:")
model_uri=f"runs:/{run_id}/model"
result = mlflow.register_model(
    model_uri , model_name
  #  "runs:/aa7c60a72db34d358f94a642831221b2/sklearn-model", "sk-learn-random-forest-reg"
)




enter run id: aa7c60a72db34d358f94a642831221b2


Successfully registered model 'XGB-Smote'.
2025/09/03 18:11:07 WARNING mlflow.tracking._model_registry.fluent: Run with id aa7c60a72db34d358f94a642831221b2 has no artifacts at artifact path 'model', registering model based on models:/m-9936811b73624efcaea4120ac95d8e9d instead
2025/09/03 18:11:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


## load the model

In [33]:
model_version=1
model_uri=f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred=loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [34]:
model_version=1
model_uri=f"models:/{model_name}@challenger"  #using TAG , so dont have to remember version number

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred=loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### transition from dev env to production

In [37]:
dev_model_uri=f"models:/{model_name}@challenger"
prod_model='anomaly-detection-prod'
client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri,dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1756909070183, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1756909070183, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='aa7c60a72db34d358f94a642831221b2', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [38]:

model_uri=f"models:/{prod_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred=loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])